In [4]:
%run script_db.py

       course_id                     course_name    subject_name course_type  \
0             16           Годовой 2к25 стандарт  Обществознание     Годовой   
1             21           Годовой 2к25 стандарт           Химия     Годовой   
2             16           Годовой 2к25 стандарт  Обществознание     Годовой   
3             25           Годовой 2к25 стандарт        Биология     Годовой   
4             19           Годовой 2к25 стандарт         Русский     Годовой   
...          ...                             ...             ...         ...   
11241         89                Вычисляй как Бог      Математика    Спецкурс   
11242         43           Химическое равновесие           Химия    Спецкурс   
11243         38               Годовой курс 2к25  Математика ОГЭ     Годовой   
11244         43           Химическое равновесие           Химия    Спецкурс   
11245         55  Скорая помощь по биологии 2к24        Биология    Спецкурс   

      course_start_date  user_id user_l

In [5]:
# Загрузка необходимых библиотек
import pandas as pd
import duckdb
import numpy as np

# Подключаемся к базе данных и выполняем SQL-запрос для извлечения данных
con = duckdb.connect()

# SQL-запрос для извлечения информации о пользователях на годовых курсах
query = """
WITH homework_count AS (
    SELECT 
        homework_done.user_id,
        lessons.course_id,
        COUNT(homework_done.id) AS homework_completed
    FROM 
        homework_done
    JOIN 
        homework ON homework_done.homework_id = homework.id
    JOIN 
        homework_lessons ON homework.id = homework_lessons.homework_id
    JOIN 
        lessons ON homework_lessons.lesson_id = lessons.id
    GROUP BY 
        homework_done.user_id, lessons.course_id
)

SELECT 
    courses.id AS course_id,                              -- ID курса
    courses.name AS course_name,                          -- Название курса
    subjects.name AS subject_name,                        -- Название предмета
    course_types.name AS course_type,                     -- Тип курса
    courses.starts_at AS course_start_date,               -- Дата старта курса
    users.id AS user_id,                                  -- ID ученика
    users.last_name AS user_last_name,                    -- Фамилия ученика
    cities.name AS city_name,                             -- Город ученика
    course_users.active AS is_active,                     -- Ученик не отчислен с курса
    course_users.created_at AS course_open_date,          -- Дата открытия курса ученику
    course_users.available_lessons AS available_lessons,  -- Количество доступных уроков в месяц
    COALESCE(homework_count.homework_completed, 0) AS homework_completed -- Число сданных ДЗ ученика на курсе
FROM 
    course_users
JOIN 
    users ON course_users.user_id = users.id               -- Присоединение таблицы пользователей
JOIN 
    courses ON course_users.course_id = courses.id         -- Присоединение таблицы курсов
JOIN 
    course_types ON courses.course_type_id = course_types.id -- Присоединение таблицы типов курсов
JOIN 
    subjects ON courses.subject_id = subjects.id           -- Присоединение таблицы предметов
JOIN 
    cities ON users.city_id = cities.id                    -- Присоединение таблицы городов
LEFT JOIN 
    homework_count ON homework_count.user_id = users.id 
    AND homework_count.course_id = courses.id              -- Присоединение с количеством сданных домашних заданий
WHERE 
    course_users.active = TRUE                            -- Только активные ученики
GROUP BY 
    courses.id, courses.name, subjects.name, course_types.name, courses.starts_at,
    users.id, users.last_name, cities.name, course_users.active, 
    course_users.created_at, course_users.available_lessons, homework_count.homework_completed;
"""

# Выполнение SQL-запроса и сохранение результата в DataFrame
df = con.execute(query).fetchdf()

# Просмотр первых строк набора данных
df.head()

,course_id,course_name,subject_name,course_type,course_start_date,user_id,user_last_name,city_name,is_active,course_open_date,available_lessons,homework_completed
0,16,Годовой 2к25 стандарт,Обществознание,Годовой,2024-09-16,10009,Чернобель,Минеральные Воды,1,2024-07-26 06:05:09,20.0,16
1,24,В погоне за пятеркой 2К25 стандарт,Математика (база),Годовой,2024-09-17,10017,Парфёнова,Хабаровск,1,2024-07-26 10:55:16,20.0,6
2,25,Годовой 2к25 стандарт,Биология,Годовой,2024-09-17,10060,Горбунова,Куйбышев,1,2024-07-27 16:06:39,28.0,7
3,24,В погоне за пятеркой 2К25 стандарт,Математика (база),Годовой,2024-09-17,10067,Сулейманова,Нефтеюганск,1,2024-07-27 08:17:46,30.0,5
4,37,Годовой курс 2к25,Русский ОГЭ,Годовой,2024-09-25,10068,Боженова,Коркино,1,2024-07-27 08:18:58,20.0,14


In [6]:
# Проверка на дубликаты
duplicates = df.duplicated().sum()
print(f"Число дубликатов: {duplicates}")

# Удаление дубликатов, если они найдены
if duplicates > 0:
    df = df.drop_duplicates()

# Проверка на пропуски в данных
missing_values = df.isnull().sum()
print("Пропуски в данных:\n", missing_values)

# Заполнение или удаление пропусков в зависимости от типа данных
# Например, если 'available_lessons' или 'homework_completed' имеют пропуски
df['available_lessons'] = df['available_lessons'].fillna(0)
df['homework_completed'] = df['homework_completed'].fillna(0)

# Проверка типов данных и приведение их к нужным типам
df['course_start_date'] = pd.to_datetime(df['course_start_date'])
df['course_open_date'] = pd.to_datetime(df['course_open_date'])

# Поиск аномальных значений (например, отрицательных значений)
# Предполагается, что все числовые значения должны быть >= 0
numerical_columns = ['available_lessons', 'homework_completed']
for col in numerical_columns:
    if (df[col] < 0).any():
        print(f"Аномальные значения в столбце {col}.")
        # Исправление: замена на 0 или другое подходящее значение
        df[col] = df[col].apply(lambda x: max(x, 0))

# Просмотр итоговых данных после предобработки
df.info()
df.head()

Число дубликатов: 0
Пропуски в данных:
 course_id                0
course_name              0
subject_name             0
course_type              0
course_start_date        0
user_id                  0
user_last_name           0
city_name                0
is_active                0
course_open_date         0
available_lessons     5567
homework_completed       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11246 entries, 0 to 11245
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   course_id           11246 non-null  int64         
 1   course_name         11246 non-null  object        
 2   subject_name        11246 non-null  object        
 3   course_type         11246 non-null  object        
 4   course_start_date   11246 non-null  datetime64[ns]
 5   user_id             11246 non-null  int64         
 6   user_last_name      11246 non-null  object        
 7   city_

,course_id,course_name,subject_name,course_type,course_start_date,user_id,user_last_name,city_name,is_active,course_open_date,available_lessons,homework_completed
0,16,Годовой 2к25 стандарт,Обществознание,Годовой,2024-09-16,10009,Чернобель,Минеральные Воды,1,2024-07-26 06:05:09,20.0,16
1,24,В погоне за пятеркой 2К25 стандарт,Математика (база),Годовой,2024-09-17,10017,Парфёнова,Хабаровск,1,2024-07-26 10:55:16,20.0,6
2,25,Годовой 2к25 стандарт,Биология,Годовой,2024-09-17,10060,Горбунова,Куйбышев,1,2024-07-27 16:06:39,28.0,7
3,24,В погоне за пятеркой 2К25 стандарт,Математика (база),Годовой,2024-09-17,10067,Сулейманова,Нефтеюганск,1,2024-07-27 08:17:46,30.0,5
4,37,Годовой курс 2к25,Русский ОГЭ,Годовой,2024-09-25,10068,Боженова,Коркино,1,2024-07-27 08:18:58,20.0,14


In [7]:
# Функция для классификации волны
def assign_wave(row):
    days_diff = (row['course_open_date'] - row['course_start_date']).days
    if days_diff <= 0:
        return 0  # 0 волна
    elif days_diff <= 7:
        return 1  # 1 волна
    elif days_diff <= 14:
        return 2  # 2 волна
    elif days_diff <= 21:
        return 3  # 3 волна
    elif days_diff <= 28:
        return 4  # 4 волна
    else:
        return 5  # 5 волна

# Применение функции к DataFrame для создания нового столбца 'wave'
df['wave'] = df.apply(assign_wave, axis=1)

# Проверка распределения по волнам
print(df['wave'].value_counts())

# Вывод первых строк с новой классификацией
df.head()

wave
0    5333
1    2343
2    1258
3     981
4     956
5     375
Name: count, dtype: int64


,course_id,course_name,subject_name,course_type,course_start_date,user_id,user_last_name,city_name,is_active,course_open_date,available_lessons,homework_completed,wave
0,16,Годовой 2к25 стандарт,Обществознание,Годовой,2024-09-16,10009,Чернобель,Минеральные Воды,1,2024-07-26 06:05:09,20.0,16,0
1,24,В погоне за пятеркой 2К25 стандарт,Математика (база),Годовой,2024-09-17,10017,Парфёнова,Хабаровск,1,2024-07-26 10:55:16,20.0,6,0
2,25,Годовой 2к25 стандарт,Биология,Годовой,2024-09-17,10060,Горбунова,Куйбышев,1,2024-07-27 16:06:39,28.0,7,0
3,24,В погоне за пятеркой 2К25 стандарт,Математика (база),Годовой,2024-09-17,10067,Сулейманова,Нефтеюганск,1,2024-07-27 08:17:46,30.0,5,0
4,37,Годовой курс 2к25,Русский ОГЭ,Годовой,2024-09-25,10068,Боженова,Коркино,1,2024-07-27 08:18:58,20.0,14,0


In [8]:
# Сохранение обработанных данных в новый CSV файл (если необходимо)
df.to_csv('processed_students_data.csv', index=False)

# Отображение результата
print(df.head())

   course_id                         course_name       subject_name  \
0         16               Годовой 2к25 стандарт     Обществознание   
1         24  В погоне за пятеркой 2К25 стандарт  Математика (база)   
2         25               Годовой 2к25 стандарт           Биология   
3         24  В погоне за пятеркой 2К25 стандарт  Математика (база)   
4         37                   Годовой курс 2к25        Русский ОГЭ   

  course_type course_start_date  user_id user_last_name         city_name  \
0     Годовой        2024-09-16    10009      Чернобель  Минеральные Воды   
1     Годовой        2024-09-17    10017      Парфёнова         Хабаровск   
2     Годовой        2024-09-17    10060      Горбунова          Куйбышев   
3     Годовой        2024-09-17    10067    Сулейманова       Нефтеюганск   
4     Годовой        2024-09-25    10068       Боженова           Коркино   

   is_active    course_open_date  available_lessons  homework_completed  wave  
0          1 2024-07-26 06:05: